<a href="https://colab.research.google.com/github/ccstevie/nhl_model/blob/main/NHL_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
from datetime import date, timedelta
from functools import reduce

In [2]:
# we will use the last 30 days as our data range
today = date.today()
print("Today's date:", today)
start = today - timedelta(days=30)
print("30 days ago:", start)

Today's date: 2024-03-06
30 days ago: 2024-02-05


In [3]:
url = f"https://www.naturalstattrick.com/teamtable.php?fromseason=20232024&thruseason=20232024&stype=2&sit=5v5&score=all&rate=n&team=all&loc=B&gpf=410&fd={start}&td={today}"
req = requests.get(url)
req.status_code

200

In [4]:
df = pd.read_html(url, header=0, index_col = 0, na_values=["-"])[0]
df.head()

,Team,GP,TOI,W,L,OTL,ROW,Points,Point %,CF,...,LDSA,LDSF%,LDGF,LDGA,LDGF%,LDSH%,LDSV%,SH%,SV%,PDO
1,Colorado Avalanche,14,683:08,6,6,2,6,14,0.500,684,...,167,49.39,5,5,50.00,3.07,97.01,8.19,93.20,1.014
2,New York Rangers,13,616:03,10,2,1,10,21,0.808,578,...,154,47.62,10,3,76.92,7.14,98.05,8.54,94.69,1.032
3,New York Islanders,12,600:35,7,3,2,7,16,0.667,549,...,130,50.94,4,6,40.00,2.96,95.38,9.57,91.89,1.015
4,Toronto Maple Leafs,14,707:46,10,4,0,9,20,0.714,718,...,129,53.76,6,3,66.67,4.00,97.67,10.25,89.93,1.002
5,Calgary Flames,12,570:52,8,4,0,8,16,0.667,627,...,129,51.32,6,4,60.00,4.41,96.90,9.44,91.49,1.009


In [5]:
# sort by CF% GF% XGF% HDCF% SH% and obtain rankings for each team
cf = df.sort_values(by="CF%", ascending=False, ignore_index=True)
cf.index += 1
cf.reset_index(inplace=True)
cf = cf.rename(columns = {"index":"CF%"})
cf = cf[["Team", "CF%"]]
# print cf as an example
cf

,Team,CF%,CF%
0,Carolina Hurricanes,1,57.94
1,Edmonton Oilers,2,57.13
2,Vancouver Canucks,3,54.81
3,Florida Panthers,4,53.89
4,Philadelphia Flyers,5,52.98
5,New Jersey Devils,6,52.73
6,Arizona Coyotes,7,52.41
7,Calgary Flames,8,52.25
8,Dallas Stars,9,52.25
9,Nashville Predators,10,51.85


In [6]:
gf = df.sort_values(by="GF%", ascending=False, ignore_index=True)
gf.index += 1
gf.reset_index(inplace=True)
gf = gf.rename(columns = {"index":"GF%"})
gf = gf[["Team", "GF%"]]

In [7]:
xgf = df.sort_values(by="xGF%", ascending=False, ignore_index=True)
xgf.index += 1
xgf.reset_index(inplace=True)
xgf = xgf.rename(columns = {"index":"xGF%"})
xgf = xgf[["Team", "xGF%"]]

In [8]:
hdcf = df.sort_values(by="HDCF%", ascending=False, ignore_index=True)
hdcf.index += 1
hdcf.reset_index(inplace=True)
hdcf = hdcf.rename(columns = {"index":"HDCF%"})
hdcf = hdcf[["Team", "HDCF%"]]

In [9]:
sh = df.sort_values(by="SH%", ascending=False, ignore_index=True)
sh.index += 1
sh.reset_index(inplace=True)
sh = sh.rename(columns = {"index":"SH%"})
sh = sh[["Team", "SH%"]]

In [10]:
# merge all DataFrames into one
dfs = [cf, gf, xgf, hdcf, sh]
final_df = reduce(lambda  left,right: pd.merge(left,right,on=['Team'],
                                            how='outer'), dfs)
final_df

,Team,CF%,CF%,GF%,GF%,xGF%,xGF%,HDCF%,HDCF%,SH%,SH%
0,Carolina Hurricanes,1,57.94,2,60.98,2,57.04,2,57.41,25,8.01
1,Edmonton Oilers,2,57.13,6,56.06,1,58.78,1,63.75,13,9.20
2,Vancouver Canucks,3,54.81,20,50.00,5,54.41,3,56.86,14,8.90
3,Florida Panthers,4,53.89,1,65.85,4,54.64,11,52.33,27,7.67
4,Philadelphia Flyers,5,52.98,15,50.94,9,53.49,8,53.16,23,8.18
5,New Jersey Devils,6,52.73,23,48.48,6,54.34,9,53.13,24,8.04
6,Arizona Coyotes,7,52.41,27,44.83,10,52.36,16,50.21,19,8.39
7,Calgary Flames,8,52.25,11,52.94,15,51.08,22,48.11,9,9.44
8,Dallas Stars,9,52.25,12,52.46,7,54.14,6,54.24,17,8.40
9,Nashville Predators,10,51.85,3,59.26,23,48.63,23,48.06,3,10.60


In [11]:
# ex. get a team's stats
final_df.loc[final_df["Team"] == "Toronto Maple Leafs"]

,Team,CF%,CF%,GF%,GF%,xGF%,xGF%,HDCF%,HDCF%,SH%,SH%
14,Toronto Maple Leafs,15,50.71,5,56.06,8,54.01,5,55.47,4,10.25


In [12]:
from get_todays_games import getGames

matchups = getGames()
matchups

[('Sabres', 'Maple Leafs'), ('Red Wings', 'Avalanche'), ('Senators', 'Ducks')]

In [13]:
res = pd.DataFrame()

for away, home in matchups:
    away_df = final_df[final_df["Team"].str.contains(away)]
    home_df = final_df[final_df["Team"].str.contains(home)]
    matchup_df = pd.concat([away_df, home_df], ignore_index=True)
    res = pd.concat([res, matchup_df], ignore_index=True)
res


,Team,CF%,CF%,GF%,GF%,xGF%,xGF%,HDCF%,HDCF%,SH%,SH%
0,Buffalo Sabres,11,51.78,9,53.85,20,49.06,13,51.69,29,6.95
1,Toronto Maple Leafs,15,50.71,5,56.06,8,54.01,5,55.47,4,10.25
2,Detroit Red Wings,25,48.14,16,50.94,27,44.86,25,45.41,2,10.80
3,Colorado Avalanche,18,49.71,8,55.77,17,49.44,12,52.22,22,8.19
4,Ottawa Senators,22,48.89,26,45.28,21,48.98,19,48.92,20,8.36
5,Anaheim Ducks,29,43.90,29,41.30,30,40.57,31,36.41,10,9.36


In [14]:
f = open("result.csv", 'w')

for col in res.columns.values:
    f.write(col + ",")

f.write("\n")

i = 0
for col in res.values:
    for row in col:
        f.write(str(row) + ",")
    if i % 2 == 0:
        f.write("\n")
    else:
        f.write("\n\n")
    i += 1

f.close()